# Regression Model B: Field 1

## Install libraries

In [1]:
install.packages('readxl')
install.packages('corrplot')
install.packages('mice')
install.packages('RColorBrewer')
install.packages('car')
install.packages('Metrics')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("readxl"):
“installation of package ‘readxl’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("mice"):
“installation of package ‘mice’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


## Call libraries

In [2]:
library(readxl)
library(corrplot)
library(mice)
library(RColorBrewer)
library(car)
library(Metrics)

corrplot 0.84 loaded
Loading required package: lattice
Loading required package: carData


## Import data

In [3]:
d1=read_excel("variable2.xlsx",sheet = 1)
d2=read_excel("variable2.xlsx",sheet = 2)
d3=read_excel("variable2.xlsx",sheet = 3) 
d4=read_excel("variable2.xlsx",sheet = 4)
d5=read_excel("variable2.xlsx",sheet = 5) 

## Field 1 (F1)

Call the data according to the Field. 

In [4]:
dE1=d1[which(d1$`ESTATE CODE`=="E1" ),]
dE2=d2[(d2$Diff ),]
dE3=d3[which(d3$Estate=="E1" ),]
dE4=d4[which(d4$Estate=="E1" ),]
dE5=d5[which(d5$Estate=="E1" ),]

## Variables

Declare the variables. The variables in which pruning time, pruning cost, manuring time, manusring cost, P&D time and P&D cost contains zero values. In this case, we assume that the data is not available (NA). Then, we construct a new table for easy representation. 

In [5]:
year=dE1$`CALENDER YEAR`
month=dE1$`CALENDER MONTH`
yield=dE1$`CPB TONNAGE`
rainfall=dE1$RAINFALL

foliar=d2$Diff

prun.time=dE3$Frequency
prun.time[prun.time == 0] <- NA

prun.cost=dE3$PruningCost
prun.cost[prun.cost == 0] <- NA

manuring.time=dE4$Frequency
manuring.time[manuring.time == 0] <- NA

manuring.cost=dE4$ManuringCost
manuring.cost[manuring.cost == 0] <- NA

pd.time=dE5$Frequency
pd.time[pd.time == 0] <- NA

pd.cost=dE5$PDCost
pd.cost[pd.cost == 0] <- NA


df1=data.frame(year,month,yield,rainfall,foliar,prun.time,prun.cost,
               manuring.time,manuring.cost,pd.time,pd.cost)

## Identify missing values in each variables

In this case, we impute the missing values. Imputation is the process of replacing missing data with substituted values.  

This study use MICE (Multivariate Imputation via Chained Equations). MICE (Multivariate Imputation via Chained Equations) is one of the commonly used package by R users. Creating multiple imputations as compared to a single imputation (such as mean) takes care of uncertainty in missing values.

MICE assumes that the missing data are Missing at Random (MAR), which means that the probability that a value is missing depends only on observed value and can be predicted using them. It imputes data on a variable by variable basis by specifying an imputation model per variable.


The mice package has function known as md.pattern(). It returns a tabular for of missing value present in each variable in a data set.

In [6]:
md.pattern(df1)

,year,month,yield,rainfall,foliar,pd.time,pd.cost,prun.time,prun.cost,manuring.time,manuring.cost,
11,1,1,1,1,1,1,1,1,1,1,1,0
8,1,1,1,1,1,1,1,0,0,1,1,2
8,1,1,1,1,1,1,1,1,1,0,0,2
9,1,1,1,1,1,0,0,1,1,1,1,2
4,1,1,1,1,1,1,1,0,0,0,0,4
2,1,1,1,1,1,0,0,0,0,1,1,4
2,1,1,1,1,1,0,0,1,1,0,0,4
,0,0,0,0,0,13,13,14,14,14,14,82


## Imputation

Here is an explanation of the parameters used:  
m  – Refers to 5 imputed data sets.  
maxit – Refers to no. of iterations taken to impute missing values.  
method – Refers to method used in imputation. We used predictive mean matching.

In [7]:
imputed_Data <- mice(df1[,-1:-4], m=5, maxit = 50, method = 'pmm', seed = 500)
summary(imputed_Data)


 iter imp variable
  1   1  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   2  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   3  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   4  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   5  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   1  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   2  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   3  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   4  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   5  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  3   1  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  3   2  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  3   3  prun.time  prun.cost  manuring.time

Multiply imputed data set
Call:
mice(data = df1[, -1:-4], m = 5, method = "pmm", maxit = 50, 
    seed = 500)
Number of multiple imputations:  5
Missing cells per column:
       foliar     prun.time     prun.cost manuring.time manuring.cost 
            0            14            14            14            14 
      pd.time       pd.cost 
           13            13 
Imputation methods:
       foliar     prun.time     prun.cost manuring.time manuring.cost 
        "pmm"         "pmm"         "pmm"         "pmm"         "pmm" 
      pd.time       pd.cost 
        "pmm"         "pmm" 
VisitSequence:
    prun.time     prun.cost manuring.time manuring.cost       pd.time 
            2             3             4             5             6 
      pd.cost 
            7 
PredictorMatrix:
              foliar prun.time prun.cost manuring.time manuring.cost pd.time
foliar             0         0         0             0             0       0
prun.time          1         0         1           

Since there are 5 imputed data sets, we select any using complete() function.
Get complete data (2nd out of 5)

Get the complete data after impute the missing value.

In [8]:
completeData <- complete(imputed_Data,2)
df2=data.frame(df1[,1:4],completeData)

## Correlation

Check the correlations with all variables. 

In [9]:
r1=cor(df2[,-1:-2])

corrplot(r1, method="number",type="upper",col=brewer.pal(n=8, name="RdYlBu"))
corrplot(r1, type="upper",col=brewer.pal(n=8, name="RdYlBu"))


## Regression

Fit the data using multiple regression for all variables. 

In [10]:
k1=lm(yield~rainfall+foliar+prun.time+prun.cost+
        manuring.time+manuring.cost+pd.time+pd.cost,data=df2)
summary(k1)
plot(k1,which=1)
plot(k1,which=2)


Call:
lm(formula = yield ~ rainfall + foliar + prun.time + prun.cost + 
    manuring.time + manuring.cost + pd.time + pd.cost, data = df2)

Residuals:
    Min      1Q  Median      3Q     Max 
-76.640 -34.931  -4.384  24.215 105.647 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)   
(Intercept)    1.503e+02  4.519e+01   3.327  0.00207 **
rainfall       1.233e-01  1.052e-01   1.172  0.24921   
foliar         9.846e-02  2.691e+00   0.037  0.97102   
prun.time      2.673e+00  8.556e+00   0.312  0.75656   
prun.cost      2.157e-03  2.850e-03   0.757  0.45418   
manuring.time -3.399e+01  3.353e+01  -1.014  0.31768   
manuring.cost  6.244e-04  9.222e-04   0.677  0.50281   
pd.time        6.647e+00  1.349e+01   0.493  0.62535   
pd.cost       -6.008e-03  6.940e-03  -0.866  0.39257   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 51.94 on 35 degrees of freedom
Multiple R-squared:  0.1891,	Adjusted R-squared:  0.003768 
F-stati

## Significant variable 

Check the significant variables using two ways. 

a.	anova  
Choose the variables that has significant value in which p<0.1.

b.	using stepwise method  
In order to avoid the multicollinearity issues in variables selection, we take into account the variance inflation factors (VIF). We remove the variables that has VIF greater than 4. Then we rebuild the model. 

The stepwise method incorporate with VIF factor is better than anova method in order to select the significant variables. 

### Using ANOVA

In [11]:
anova(k1)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
rainfall,1,12182.88068,12182.88068,4.51546290,0.04072502
foliar,1,55.49196,55.49196,0.02056754,0.88678584
prun.time,1,55.43328,55.43328,0.02054579,0.88684530
prun.cost,1,2559.81786,2559.81786,0.94877089,0.33671667
manuring.time,1,4379.05295,4379.05295,1.62305219,0.21106174
manuring.cost,1,586.17619,586.17619,0.21726034,0.64402336
pd.time,1,182.42064,182.42064,0.06761238,0.79636900
pd.cost,1,2021.81672,2021.81672,0.74936615,0.39257223
Residuals,35,94431.25400,2698.03583,NA,NA


### Using stepwise

In [12]:
selectedMod <- step(k1)
summary(selectedMod)

##Avoid Multicollinearity and Statistical Significance
##Recursively remove variables with variance inflation factors: VIF > 4
# Remove vars with VIF> 4 and re-build model until none of VIFs don't exceed 4.
all_vifs <- car::vif(selectedMod)
print(all_vifs)

while(any(all_vifs > 4)){
  var_with_max_vif <- names(which(all_vifs == max(all_vifs)))  # get the var with max vif
  signif_all <- signif_all[!(signif_all) %in% var_with_max_vif]  # remove
  myForm <- as.formula(paste("ozone_reading ~ ", paste (signif_all, collapse=" + "), sep=""))  # new formula
  selectedMod <- lm(myForm, data=inputData)  # re-build model with new formula
  all_vifs <- car::vif(selectedMod)
}

summary(selectedMod)

Start:  AIC=355.54
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    manuring.cost + pd.time + pd.cost

                Df Sum of Sq   RSS    AIC
- foliar         1       3.6 94435 353.54
- prun.time      1     263.4 94695 353.67
- pd.time        1     654.8 95086 353.85
- manuring.cost  1    1236.8 95668 354.12
- prun.cost      1    1545.6 95977 354.26
- pd.cost        1    2021.8 96453 354.48
- manuring.time  1    2772.5 97204 354.82
- rainfall       1    3704.4 98136 355.24
<none>                       94431 355.54

Step:  AIC=353.54
yield ~ rainfall + prun.time + prun.cost + manuring.time + manuring.cost + 
    pd.time + pd.cost

                Df Sum of Sq   RSS    AIC
- prun.time      1     260.1 94695 351.67
- pd.time        1     651.2 95086 351.85
- manuring.cost  1    1491.6 95926 352.23
- prun.cost      1    1543.9 95979 352.26
- pd.cost        1    2086.2 96521 352.51
- manuring.time  1    2923.9 97359 352.89
- rainfall       1    3884.3 98319 353.32


Call:
lm(formula = yield ~ rainfall, data = df2)

Residuals:
    Min      1Q  Median      3Q     Max 
-76.957 -33.932  -1.002  27.616 113.793 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 123.66338   17.58804   7.031 1.31e-08 ***
rainfall      0.17404    0.07857   2.215   0.0322 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 49.83 on 42 degrees of freedom
Multiple R-squared:  0.1046,	Adjusted R-squared:  0.0833 
F-statistic: 4.907 on 1 and 42 DF,  p-value: 0.03222


ERROR: Error in vif.default(selectedMod): model contains fewer than 2 terms


## Prediction

Predict with the resample values.

In [13]:
set.seed(123)
df3=sample(1:nrow(df2),12)
df4=df2[df3,]
estimate=predict(selectedMod,newdata=df4)
estimate
sum.estimate=sum(estimate)
sum.estimate

df5=data.frame(estimate,sum.estimate)

13       34       18       37       38        2       21       43 
163.8667 137.4126 146.1146 170.8283 145.4184 145.9405 163.6927 174.4832 
      20       16       33       15 
164.5629 139.3270 187.7102 159.6897

[1] 1899.047